In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8118, train_loss: 0.32644224, train_accuracy: 0.3750, test_Accuracy: 0.2346
Epoch: [ 0] [    1/  468] time: 1.1365, train_loss: 0.40490192, train_accuracy: 0.4531, test_Accuracy: 0.3428
Epoch: [ 0] [    2/  468] time: 1.4805, train_loss: 0.36095071, train_accuracy: 0.4922, test_Accuracy: 0.4294
Epoch: [ 0] [    3/  468] time: 1.8307, train_loss: 0.40762043, train_accuracy: 0.5312, test_Accuracy: 0.5219
Epoch: [ 0] [    4/  468] time: 2.2376, train_loss: 0.22052749, train_accuracy: 0.5938, test_Accuracy: 0.5722
Epoch: [ 0] [    5/  468] time: 2.5418, train_loss: 0.29408538, train_accuracy: 0.6562, test_Accuracy: 0.6100
Epoch: [ 0] [    6/  468] time: 2.8469, train_loss: 0.36145312, train_accuracy: 0.6328, test_Accuracy: 0.6408
Epoch: [ 0] [    7/  468] time: 3.1491, train_loss: 0.25942695, train_accuracy: 0.6562, test_Accuracy: 0.6772
Epoch: [ 0] [    8/  468] time: 3.4503,

Epoch: [ 0] [   74/  468] time: 26.1889, train_loss: 0.23662567, train_accuracy: 0.8906, test_Accuracy: 0.9383
Epoch: [ 0] [   75/  468] time: 26.5639, train_loss: 0.19752085, train_accuracy: 0.9141, test_Accuracy: 0.9377
Epoch: [ 0] [   76/  468] time: 26.9448, train_loss: 0.11011227, train_accuracy: 0.9766, test_Accuracy: 0.9359
Epoch: [ 0] [   77/  468] time: 27.3194, train_loss: 0.07351858, train_accuracy: 0.9688, test_Accuracy: 0.9347
Epoch: [ 0] [   78/  468] time: 27.6983, train_loss: 0.12919027, train_accuracy: 0.9297, test_Accuracy: 0.9332
Epoch: [ 0] [   79/  468] time: 28.0683, train_loss: 0.19565910, train_accuracy: 0.9375, test_Accuracy: 0.9306
Epoch: [ 0] [   80/  468] time: 28.4613, train_loss: 0.11049391, train_accuracy: 0.9297, test_Accuracy: 0.9297
Epoch: [ 0] [   81/  468] time: 28.8353, train_loss: 0.06912987, train_accuracy: 0.9688, test_Accuracy: 0.9301
Epoch: [ 0] [   82/  468] time: 29.1914, train_loss: 0.12107599, train_accuracy: 0.9375, test_Accuracy: 0.9319
E

Epoch: [ 0] [  148/  468] time: 54.7695, train_loss: 0.07885216, train_accuracy: 0.9844, test_Accuracy: 0.9580
Epoch: [ 0] [  149/  468] time: 55.1685, train_loss: 0.07391743, train_accuracy: 0.9688, test_Accuracy: 0.9590
Epoch: [ 0] [  150/  468] time: 55.5544, train_loss: 0.12036243, train_accuracy: 0.9609, test_Accuracy: 0.9582
Epoch: [ 0] [  151/  468] time: 56.0132, train_loss: 0.11872225, train_accuracy: 0.9609, test_Accuracy: 0.9570
Epoch: [ 0] [  152/  468] time: 56.4032, train_loss: 0.16013511, train_accuracy: 0.9688, test_Accuracy: 0.9573
Epoch: [ 0] [  153/  468] time: 56.8221, train_loss: 0.10447432, train_accuracy: 0.9922, test_Accuracy: 0.9572
Epoch: [ 0] [  154/  468] time: 57.2090, train_loss: 0.11679763, train_accuracy: 0.9766, test_Accuracy: 0.9571
Epoch: [ 0] [  155/  468] time: 57.6110, train_loss: 0.15120855, train_accuracy: 0.9531, test_Accuracy: 0.9583
Epoch: [ 0] [  156/  468] time: 57.9919, train_loss: 0.12321943, train_accuracy: 0.9609, test_Accuracy: 0.9582
E

Epoch: [ 0] [  222/  468] time: 85.9171, train_loss: 0.11284730, train_accuracy: 0.9375, test_Accuracy: 0.9575
Epoch: [ 0] [  223/  468] time: 86.3100, train_loss: 0.13075560, train_accuracy: 0.9297, test_Accuracy: 0.9576
Epoch: [ 0] [  224/  468] time: 86.7508, train_loss: 0.03561377, train_accuracy: 0.9922, test_Accuracy: 0.9581
Epoch: [ 0] [  225/  468] time: 87.1607, train_loss: 0.07297901, train_accuracy: 0.9766, test_Accuracy: 0.9556
Epoch: [ 0] [  226/  468] time: 87.5766, train_loss: 0.10004246, train_accuracy: 0.9609, test_Accuracy: 0.9555
Epoch: [ 0] [  227/  468] time: 87.9746, train_loss: 0.08218376, train_accuracy: 0.9688, test_Accuracy: 0.9549
Epoch: [ 0] [  228/  468] time: 88.3615, train_loss: 0.10720508, train_accuracy: 0.9766, test_Accuracy: 0.9551
Epoch: [ 0] [  229/  468] time: 88.7535, train_loss: 0.06159233, train_accuracy: 0.9688, test_Accuracy: 0.9554
Epoch: [ 0] [  230/  468] time: 89.1434, train_loss: 0.05443198, train_accuracy: 0.9766, test_Accuracy: 0.9544
E

Epoch: [ 0] [  296/  468] time: 116.1546, train_loss: 0.11596017, train_accuracy: 0.9453, test_Accuracy: 0.9620
Epoch: [ 0] [  297/  468] time: 116.5526, train_loss: 0.07460530, train_accuracy: 0.9844, test_Accuracy: 0.9627
Epoch: [ 0] [  298/  468] time: 116.9695, train_loss: 0.07105274, train_accuracy: 1.0000, test_Accuracy: 0.9630
Epoch: [ 0] [  299/  468] time: 117.3685, train_loss: 0.15524851, train_accuracy: 0.9531, test_Accuracy: 0.9641
Epoch: [ 0] [  300/  468] time: 117.7658, train_loss: 0.08990064, train_accuracy: 0.9766, test_Accuracy: 0.9634
Epoch: [ 0] [  301/  468] time: 118.2127, train_loss: 0.06856830, train_accuracy: 0.9688, test_Accuracy: 0.9624
Epoch: [ 0] [  302/  468] time: 118.6116, train_loss: 0.03602609, train_accuracy: 1.0000, test_Accuracy: 0.9629
Epoch: [ 0] [  303/  468] time: 119.0424, train_loss: 0.05711953, train_accuracy: 0.9844, test_Accuracy: 0.9619
Epoch: [ 0] [  304/  468] time: 119.4543, train_loss: 0.09863304, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 146.3602, train_loss: 0.07020974, train_accuracy: 0.9688, test_Accuracy: 0.9645
Epoch: [ 0] [  371/  468] time: 146.7491, train_loss: 0.04084216, train_accuracy: 0.9688, test_Accuracy: 0.9647
Epoch: [ 0] [  372/  468] time: 147.1361, train_loss: 0.09060808, train_accuracy: 0.9766, test_Accuracy: 0.9627
Epoch: [ 0] [  373/  468] time: 147.5354, train_loss: 0.02485177, train_accuracy: 0.9766, test_Accuracy: 0.9622
Epoch: [ 0] [  374/  468] time: 147.9293, train_loss: 0.08182291, train_accuracy: 0.9766, test_Accuracy: 0.9626
Epoch: [ 0] [  375/  468] time: 148.3296, train_loss: 0.08229029, train_accuracy: 0.9766, test_Accuracy: 0.9629
Epoch: [ 0] [  376/  468] time: 148.7193, train_loss: 0.10044748, train_accuracy: 0.9609, test_Accuracy: 0.9630
Epoch: [ 0] [  377/  468] time: 149.1292, train_loss: 0.06574000, train_accuracy: 0.9688, test_Accuracy: 0.9632
Epoch: [ 0] [  378/  468] time: 149.5371, train_loss: 0.06609713, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 179.1042, train_loss: 0.10397348, train_accuracy: 0.9688, test_Accuracy: 0.9674
Epoch: [ 0] [  445/  468] time: 179.5321, train_loss: 0.08754227, train_accuracy: 0.9766, test_Accuracy: 0.9673
Epoch: [ 0] [  446/  468] time: 179.9629, train_loss: 0.05286007, train_accuracy: 0.9922, test_Accuracy: 0.9666
Epoch: [ 0] [  447/  468] time: 180.3748, train_loss: 0.05373925, train_accuracy: 0.9766, test_Accuracy: 0.9656
Epoch: [ 0] [  448/  468] time: 180.7838, train_loss: 0.06947993, train_accuracy: 0.9609, test_Accuracy: 0.9658
Epoch: [ 0] [  449/  468] time: 181.3103, train_loss: 0.05721885, train_accuracy: 0.9844, test_Accuracy: 0.9654
Epoch: [ 0] [  450/  468] time: 181.8160, train_loss: 0.10717013, train_accuracy: 0.9688, test_Accuracy: 0.9653
Epoch: [ 0] [  451/  468] time: 182.2778, train_loss: 0.05318617, train_accuracy: 0.9922, test_Accuracy: 0.9651
Epoch: [ 0] [  452/  468] time: 182.7276, train_loss: 0.06814774, train_accuracy: 0.9844, test_Accuracy: